In [1]:
import re

import torch
import requests
import pandas as pd
import transformers
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

In [2]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device);

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
def avg_pooling(window: torch.Tensor, seq_len: torch.Tensor, key_padding_mask: torch.Tensor)  -> torch.FloatTensor:
    """Помогает пулить padded последовательности
    
    source: https://github.com/iorymaeda/ArcDOTA/blob/master/utils/nn/prematch.py 
    
    :param window: - window/array to pool
    | window | : (batch_size, seq_len, d_model)
    
    :param seq_len: - array with seq_len per sample in batch
    | seq_len | : (batch_size)
    
    :param key_padding_mask: - array with padded mask
    | key_padding_mask | : (batch_size, seq_len)
    
    """
    
    # multiply window by mask - zeros all padded tokens
    pooled = torch.mul(window, key_padding_mask.unsqueeze(2))
    # |pooled| : (batch_size, seq_len, d_model)

    # sum all elements by seq_len dim
    pooled = pooled.sum(dim=1)
    # |pooled| : (batch_size, d_model)

    # divide samples by by its seq_len, so we will get mean values by each sample
    pooled = pooled / seq_len.unsqueeze(1)
    # |pooled| : (batch_size, d_model)

    return pooled

In [4]:
def split(text: str) -> str:
    return text.replace("-", " ").replace("_", " ").split()

In [5]:
def crop_text_to_patches(text: str, slice_range:int=2) -> list:
    query_slices = []
    query_words = split(text)
    
    if len(query_words) < slice_range:
        query_slices.append(text)
        
    else:
        for batch in range(0, len(query_words)-(slice_range-1)):
            s = ""
            for _ in range(slice_range):
                s+= query_words[batch + _] + " "
            s = s.strip()

            query_slices.append( s )
            
    return query_slices

crop_text_to_patches('Привет мир, меня зовут Ангелина', slice_range=3)

['Привет мир, меня', 'мир, меня зовут', 'меня зовут Ангелина']

In [6]:
def dict_to_device(d, device):
    return {k: v.to(device) for k, v in d.items()}

\\(\mathbf {cos\_similarity} =\|\mathbf {a} \|\ \|\mathbf {b} \|\cos \theta \\)

In [7]:
@torch.no_grad()
def classify(query: str, anchor: list, slice_range=2) -> float:
    query = query.lower()
    query_slices = crop_text_to_patches(query, slice_range=slice_range)
    
    text = anchor + query_slices
    tokens = tokenizer(text, return_tensors='pt', padding=True)
    out = model(**dict_to_device(tokens, device))
    out = dict_to_device(out, 'cpu')
    
    # embs = out['last_hidden_state'].mean(dim=1)
    embs = avg_pooling(out['last_hidden_state'], tokens['attention_mask'].sum(1), tokens['attention_mask'])
    normed = embs / torch.norm(embs, dim=1, keepdim=True)
    
    _anchor, _query = normed[:len(anchor)], normed[len(anchor):]
    max_ = (_anchor @ _query.T).max()
    
    if slice_range == 1:
        return max_
    else:
        return max(max_, classify(query, anchor, slice_range-1))

In [29]:
def to_camel_case(text):
    if len(text) == 0: return text

    s = split(text)
    output = ""
    for _t in s:
        output += _t.capitalize()
        output += " "
    return output.rstrip()

to_camel_case('NER модель не умеет работать с именнованными сущностями начинающимися с нижнего регистра :(')

'Ner Модель Не Умеет Работать С Именнованными Сущностями Начинающимися С Нижнего Регистра :('

In [33]:
def preproc(replic: str) -> dict[str, list[int]]:
    # Кропаем и подаём патчами т.к. модель глупенькая и на больших предложениях путаеться 
    return crop_text_to_patches(to_camel_case(replic), slice_range=5) 

def get_ner(replic: str): 
    ner = session.post(
        url="http://127.0.0.1:8201/model", 
        json={ "x": preproc(replic)}
    ).json()
    
    # Куча постпроцессинга
    ners = [[] for _ in range(len(split(replic)))]
    words = []
    for idx, v in enumerate(ner):
        t, n = v

        if idx == 0: words += t
        else: words += t[-1:]

        n = [re.sub("\D-", "", ner_v) for ner_v in n]
        for jdx, ner_v in enumerate(n):
            ners[idx+jdx].append(ner_v)

    align = []
    for i in ners:
        d = {}
        for entity in i:
            if entity in d:
                d[entity] += 1
            else:
                d[entity] = 1

        for key in d:
            d[key] /= len(i)
        align.append(d)
        
    NER = []
    for t in align:
        entity = max(t, key=t.get)
        if t[entity] >= 0.5:
            NER.append(entity)
        else:
            NER.append('O')

    collected = {}
    _previous_num = 0
    _previous_tag = NER[0]
    for idx, tag in enumerate(NER):
        if idx > 0:
            if tag != _previous_tag:
                if _previous_tag not in collected:
                    collected[_previous_tag] = []

                collected[_previous_tag].append([_previous_num, idx])

                _previous_num = idx
                _previous_tag = tag

    if _previous_tag not in collected:
        collected[_previous_tag] = []

    collected[_previous_tag].append([_previous_num, len(NER)])
    
    if 'O' in collected:
        del collected['O']
    
    return collected

def extract_ner(replic: str):
    ners = get_ner(replic)
    splitted_replic = split(replic)
    for key in ners:
        for idx, pos in enumerate(ners[key]):
            s = ""
            _replic = splitted_replic[pos[0]:pos[1]]
            for word in _replic:
                s+= word
                s+= " "
            s = s.strip()

            ners[key][idx] = s
    return ners

In [35]:
session = requests.Session()
df = pd.read_csv('test_data.csv')

for idx, replic in enumerate(tqdm(df['text'])): 
    replic = replic.strip()
    
    # --------------------------------------------------------------------- #
    ners = extract_ner(replic)
    greetings = classify(replic, slice_range=2, anchor=['приветствие', 'здравствуйте', 'добрый день', 'доброе утро', 'добрый вечер'])
    farewells = classify(replic, slice_range=2, anchor=['до свидания', 'всего хорошего', 'прощай', 'прощайте', 'хорошего дня', 'хорошего вечера'])
    name = classify(replic, slice_range=3, anchor=['меня зовут', 'зовут меня'])

    greetings = greetings.item()
    farewells = farewells.item()
    name = name.item()
    
    # --------------------------------------------------------------------- #
    # speech2text съездает некоторые слова, отдельно обработаем такие случаи
    words = split(replic.lower())[0]
    if words[0] in ['алло', 'ало']: del words[0]
    
    if words[0] in ['добрый', 'доброе']:
        greetings = 0.9 if greetings < 0.9 else greetings
    
    # --------------------------------------------------------------------- #
    df.loc[idx, 'greetings_score'] = greetings
    df.loc[idx, 'farewells_score'] = farewells
    df.loc[idx, 'name_score'] = name

    for entity in ners:
        df.loc[idx, entity] = ners[entity][0]

100%|████████████████████████████████████████████████████████████████████████████████| 480/480 [01:41<00:00,  4.71it/s]


### post proccesing

In [36]:
df_copy = df.copy()

In [88]:
df = df_copy.copy()

In [89]:
# Модель очень тригерится на Алло, думает это имена
# К сожалению скоры deeppavlov не выдаёт
df.loc[df['PER'] == 'Алло', 'PER'] = float('nan')
df["PER"] = df["PER"].apply(lambda x:  x.lower().replace("алло", "") if type(x) is str else x)

In [90]:
# Это нам не надо
df.drop(['LOC'], axis=1, inplace=True)

In [91]:
# Что-то мне кажется что роль менеджера и клиента перепутаны
# Буду считать всё для клиентов представляю что это менеджеры, к тому же этот нюанс можно уточнить и легко исправить

# Ещё один пост процессинг, обычно здороваются в начале диалога, предпологаю что компанию тоже стоит назвать в начале диалога,
# а прощаются в конце, таким образом отчистим от мусора который выдал deeppavlov
role = 'client'
greetings_index = []
farewells_index = []
name_index = []

per_index = []
org_index = []

for dlg_id  in df['dlg_id'].unique():
    corpus = df[(df['dlg_id'] == dlg_id) & (df['role'] == role)]
    
    # Приветсвие обычно в начале
    slice_ = corpus.iloc[:2]['greetings_score']
    slice_ = slice_[slice_ >= 0.85]
    for index in slice_[slice_ == slice_.max()].index:
        greetings_index.append(index)
        break
        
    # Прощание обыччно в конце
    slice_ = corpus.iloc[-4:]['farewells_score']
    slice_ = slice_[slice_ >= 0.85]
    for index in slice_[slice_ == slice_.max()].index:
        farewells_index.append(index)
        break
        
    slice_ = corpus.iloc[:4]['name_score']
    slice_ = slice_[slice_ >= 0.8]
    for index in slice_[slice_ == slice_.max()].index:
        name_index.append(index)
        break
        
    slice_ = corpus.iloc[:4]['PER']
    for index in slice_.index:
        per_index.append(index)
    
    slice_ = corpus.iloc[:4]['ORG']
    for index in slice_.index:
        org_index.append(index)
        
df['greetings'] = False
df['farewells'] = False
df['name'] = False
df.loc[greetings_index, 'greetings'] = True
df.loc[farewells_index, 'farewells'] = True
df.loc[name_index, 'name'] = True

df['PER_name'] = float('nan')
df['ORG_name'] = float('nan')
df.loc[per_index, 'PER_name'] = df.loc[per_index, 'PER']
df.loc[org_index, 'ORG_name'] = df.loc[org_index, 'ORG']

In [92]:
# Собираем в словарь всё остальное

dlf_output = {}
for dlg_id  in df['dlg_id'].unique():
    df_slice = df[(df['dlg_id'] == dlg_id)]
    
    greeting = df_slice['greetings'].any()
    farewell = df_slice['farewells'].any()
    
    PER_name = df_slice[df_slice['role'] == role]['PER_name'].dropna().unique().tolist()
    ORG_name = df_slice[df_slice['role'] == role]['ORG_name'].dropna().unique().tolist()
    
    greeting_text = df_slice[df_slice['greetings']]['text']
    farewell_text = df_slice[df_slice['farewells']]['text']
    name_text = df_slice[df_slice['name']]['text']
    
    greeting_text = greeting_text.values[0] if len(greeting_text) else ''
    farewell_text = farewell_text.values[0] if len(farewell_text) else ''
    name_text = name_text.values[0] if len(name_text) else ''
    
    dlf_output[dlg_id] = {
        'greeting': greeting,
        'farewell': farewell,
        'farewell_text': farewell_text,
        'greeting_text': greeting_text,
        'name_text': name_text,
        'PER_name': PER_name,
        'ORG_name': ORG_name,
        'is_polite': (greeting and farewell)
    }

In [93]:
dlf_output

{0: {'greeting': True,
  'farewell': True,
  'farewell_text': 'Всего хорошего до свидания',
  'greeting_text': 'Алло здравствуйте',
  'name_text': 'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается',
  'PER_name': ['ангелина'],
  'ORG_name': ['компания диджитал бизнес'],
  'is_polite': True},
 1: {'greeting': True,
  'farewell': True,
  'farewell_text': 'До свидания',
  'greeting_text': 'Алло здравствуйте',
  'name_text': 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается',
  'PER_name': ['ангелина'],
  'ORG_name': ['компания диджитал бизнес'],
  'is_polite': True},
 2: {'greeting': True,
  'farewell': False,
  'farewell_text': '',
  'greeting_text': 'Алло здравствуйте',
  'name_text': 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там',
  'PER_name